In this Jupyter notebook we created an algorithm that takes a set of public securities of our choosing and finds correlations or cointegrations between them. Using this, we then mark buy or sell signals accordingly allowing us to profit using pairwise trading.  

In [117]:
#imports
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import yfinance as yf
from statsmodels.tsa.stattools import coint, adfuller
from pandas_datareader import data as pdr
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime
from scipy.stats.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

Area to choose what Stocks to track:

In [108]:
yf.pdr_override()
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()
tickers = ['TSLA', 'NKE', 'AMZN', 'WMT', 'GOOG', 'JPM', 'META', 'NVDA', 'ABBV', 'AAPL', 'ADBE', 'ORCL', 'EBAY', 'MSFT', 'QCOM', 'HPQ', 'JNPR', 'AMD', 'IBM', 'VOO']


df = pdr.get_data_yahoo(tickers, start, end)['Close']
df.tail()

[*********************100%***********************]  20 of 20 completed


,AAPL,ABBV,ADBE,AMD,AMZN,EBAY,GOOG,HPQ,IBM,JNPR,JPM,META,MSFT,NKE,NVDA,ORCL,QCOM,TSLA,VOO,WMT
Date,,,,,,,,,,,,,,,,,,,,
2023-02-06,151.729996,145.020004,375.230011,83.680000,102.180000,49.980000,103.470001,29.770000,136.179993,30.799999,141.919998,186.059998,256.769989,125.730003,210.889999,88.529999,132.929993,194.759995,376.660004,140.679993
2023-02-07,154.649994,145.139999,383.820007,85.910004,102.110001,50.169998,108.040001,30.000000,135.839996,31.219999,143.649994,191.619995,267.559998,125.330002,221.729996,87.739998,136.630005,196.809998,381.519989,140.979996
2023-02-08,151.919998,144.610001,378.359985,84.690002,100.050003,49.160000,100.000000,29.610001,135.979996,31.059999,142.639999,183.429993,266.730011,122.910004,222.050003,86.690002,132.169998,201.289993,377.339996,140.220001
2023-02-09,150.869995,148.699997,375.809998,83.209999,98.239998,48.389999,95.459999,29.660000,133.750000,30.969999,140.419998,177.919998,263.619995,122.180000,223.369995,86.650002,130.529999,207.320007,374.109985,141.520004
2023-02-10,151.009995,152.050003,370.989990,81.480003,97.610001,48.080002,94.860001,29.969999,135.600006,31.129999,141.039993,174.149994,263.100006,122.230003,212.649994,87.139999,128.990005,196.889999,375.019989,143.720001


First we need to test for stationarity using the Dickey-Fuller Test; we obviously only want to look at stocks that are not following stationarity.

In [109]:
def stationarity_test(X, cutoff=0.01):
    pvalue = adfuller(X)[1]
    if pvalue < cutoff:
        print('p-value = ' + str(pvalue) + ' The series ' + X.name +' is likely stationary.')
    else:
        print('p-value = ' + str(pvalue) + ' The series ' + X.name +' is likely non-stationary.')

This is to test if the Stationarity Test actually works:

In [110]:
ticker = yf.Ticker('AAPL')
Stock_history = ticker.history(period = "60d")
Stock_close = Stock_history["Close"]
stationarity_test(Stock_close)

p-value = 0.40673803423763844 The series Close is likely non-stationary.


Test for Cointegration/Correlated Pairs:

In [111]:
pairs = []
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return pvalue_matrix, score_matrix, pairs

In [155]:
def add_row(S1, S2, df):
    s1_name = S1.name
    s2_name = S2.name
    result = coint(S1, S2)
    score = result[0]
    p_value = result[1]
    corr = pearsonr(S1, S2)[0]
    temp_df = pd.DataFrame([s1_name, s2_name, score, p_value, corr])
    temp_df = temp_df.transpose()
    temp_df.columns = 'Stock 1', 'Stock 2', 'CoInt', 'P CoInt', 'Correlation'
    if df is not None:
        df = pd.concat([df, temp_df], ignore_index=True)
    else:
        df = temp_df
    return df


In [160]:
def make_table(data):
    temp_df = None
    n = data.shape[1]
    for i in range(n):
        for j in range(i+1, n):
            S1 = data.iloc[:,i]
            S2 = data.iloc[:,j]
            temp_df = add_row(S1, S2, temp_df)
    return temp_df



Now we will test the Cointegrated Pairs Function:

In [161]:
#find_cointegrated_pairs(df)
make_table(df)



,Stock 1,Stock 2,CoInt,P CoInt,Correlation
0,AAPL,ABBV,-2.35489,0.346507,0.869221
1,AAPL,ADBE,-1.009662,0.900999,0.887219
2,AAPL,AMD,-2.446887,0.302982,0.963639
3,AAPL,AMZN,-0.179205,0.980747,0.874103
4,AAPL,EBAY,-1.510391,0.757186,0.843622
...,...,...,...,...,...
185,QCOM,VOO,-3.047437,0.099356,0.926969
186,QCOM,WMT,-3.280106,0.057521,0.876282
187,TSLA,VOO,-2.872433,0.143722,0.910114
188,TSLA,WMT,-2.365559,0.341336,0.80888


Now we will create a Pandas Dataframe to store all of the Cointegration values and the Correlations values. We have the option to store each pair twice or once in order to create iteration through the information easier for post-processing. 

,Stock 1,Stock 2,Correlation
0,ADBE,AMZN,None
1,AMD,GOOG,None
2,AMD,MSFT,None
3,AMD,NKE,None
4,AMD,QCOM,None
5,AMD,WMT,None
6,EBAY,META,None
7,GOOG,ORCL,None
8,GOOG,TSLA,None
9,IBM,JPM,None
